In [1]:
## challenge was to convert lat/long to x,y screen positions and have the nodes be at the approx relative
# positions.
## also need to ensure that East/West, North/South is translated correctly to x/y pixel coordinate system. Mostly
## trial error, but common sense, like knowing Kenmore should be to the left of South Station on network 
## and thus smaller x value helps.
## +71.05)*10000, et al is simply for giving a more granular pixel result, since ploting without this adjustment
## gives us pixels VERY close to each other



### Libraries

In [1]:
import urllib.request, json, time
from twilio.rest import TwilioRestClient 

### Function Definitions

In [2]:
## GOOD CODE -4/26###, test again if I do TODOs

### TODO - Look at sending alerts for disabled bikes or dcoks  ###

## set a default value for map_location so we can use this for variety of texts
def send_sms_alert(message, map_location = ''):

    ## Credentials go below
    ## ACCOUNT_SID = ## Saved in environmental variables so authentication isn't hard coded in code
    ## AUTH_TOKEN = 
 
    client = TwilioRestClient() #(ACCOUNT_SID, AUTH_TOKEN) - Rest Client looks for credentials in environ variables
    client.messages.create(
        to="+18595366066", 
        from_="+16172863396",  ## this is TWILIO account phone number
        body = message + map_location)
    
        
    print('SMS message sent ', message)

##############################################################


## default msg (sent most often) will be low bikes; others added as needed
def send_alert(id, msg = 1):
    ### form the message, then call the actual function that sends it above. This function should be able to send a
    # variety of message types. After message is formed we'll pass it cleanly to the function above
    
    ## we don't need statement below but keeping as a reminder that this dict is able to be referenced whether we
    ## pass it or not as it's passed by reference not value
    global dict_bikes_master
    
    lat = dict_bikes_master[id]['lat']
    lon = dict_bikes_master[id]['lon']
    stat_name = dict_bikes_master[id]['name']
    bikes_available = dict_bikes_master[id]['num_bikes_available']
    if msg == 1:
        message_to_send = stat_name +' is low on bikes. ' + str(bikes_available) + ' available.'
        map_location = 'http://maps.apple.com/?ll='+str(lat)+','+str(lon)+'&q='+stat_name.replace(' ', '%20')
        
    ## execute Twilio message
    send_sms_alert(message_to_send, map_location)

In [3]:
## Standard code for opening URL and resturning JSON object
### does there need to be a 'with/open here?
def json_in(url):
    
    try:
        response = urllib.request.urlopen(url)
    except urllib.error.HTTPError as e1:
        print(e1)
    else:
        #use json.loads to unpack the json
        return json.loads(response.read().decode('utf-8'))
    

## we'll use a function for less code and cleanliness
def json_out(out_file_name, data, var_text):
    with open(out_file_name, mode = 'w') as f:
        f.write("var " + var_text + " =") #Kaleigh added this
        json.dump(data , f,indent = 2)

In [4]:
## GOOD CODE -4/26###

## Convert lat/long to pixels. See detailed notes
def lon_to_x(lon):
    return round((40)*(90 +(float(lon)+71.05)*10000) / 180,2)

def lat_to_y(lat):
    return round((70)*(180 -(float(lat)-42.35)*10000) / 360,2)

## shorten the name of station to first 5 words. Here for mainly cleanliness in code below        
def shorten(station):
    return' '.join(station.split( )[:5])

In [5]:
## TODO might need to set a return ???
def set_color_status(current_station):
    #print(current_station)
    bikes = current_station['num_bikes_available']
    current_station['color'] = 'red' if bikes < 3 else 'yellow' if bikes < 6 else 'green'


#### Use a print function for cell printing. This keeps our code cleaner as we are print various dictionary items and it's quite messy

In [6]:
def print_status_console(station):
    
    print('Time reported -',time.ctime(station['last_reported']))
    print(dict_bikes_master[station['station_id']]['name'])
    #### TODO lambda to fix singular/plural???
    print('Change of',current_bikes_avail - previous_bikes_avail, 'bikes')
    print(current_bikes_avail, '- currently available')
    print(previous_bikes_avail, '- previously available\n')

At time reported - Wed Apr 26 08:23:27 2017
Burlington Ave at Brookline Ave change of
-1 bikes
8 - current bikes available
9 - previous bikes available

## Constants

In [7]:
## The Two Hubway Feed URLs we need
### Name, lat/long,ID
url_station_info = 'https://gbfs.thehubway.com/gbfs/en/station_information.json'

## Online?, capacity, bikes available, last rented, etc
url_station_status = 'https://gbfs.thehubway.com/gbfs/en/station_status.json'

## When to send out low bike message
alert_number = 5

## Update our feed this often in seconds. Hard coded for now from JSON feed
ttl = 10 ## TODO eventually set this from the feed data

## Main Code Starts Here!

### This gets station position & name. Also our initial data for our baseline. We then update via the live feed in a 
### loop in the following cell. This cell creates 2 JSON feeds, one with station name and position (translated from
### lat/lon to pixels), the second with the data we want to visualize - bikes available, total activity, bikes low?.
### Also save all data into a master dictionary (might do away with that).

## Split Below up into various cells as logical

In [27]:
## get bikes available at online stations
json_bikes_avail = json_in(url_station_status)
json_station_locations = json_in(url_station_info)

### make JSON file for status (bikes avail and color code)  and start master dictionary 
## We need two JSON files for our network visualization 1) station name/ID, position 2) station name/ID, data
dict_bikes_master = {}
json_list_bikes_avail = []
json_list_stations_locations = []


## TODO ## Possibly don't need dictionary at all, just manipulate JSON object....???

## station is open so we can plot it and update the mster dict. 220 is a test station we need to exclude
for station in json_bikes_avail['data']['stations']:    
    if station['is_renting'] == 1 and station['station_id']!= '220': 
        
#         if station['num_bikes_available'] < 3:
#             dict_bikes_master[station['station_id']] = station
#             dict_bikes_master[station['station_id']]['color'] = 'red'
#         elif station['num_bikes_available'] < 6:           
#             dict_bikes_master[station['station_id']] = station
#             dict_bikes_master[station['station_id']]['color'] = 'yellow'
#         else:
#             dict_bikes_master[station['station_id']] = station
#             dict_bikes_master[station['station_id']]['color'] = 'green'

        ## easier to get all the data as opposed to just what we need
        dict_bikes_master[station['station_id']] = station
        dict_bikes_master[station['station_id']]['total_activity'] = 0
        
        set_color_status(dict_bikes_master[station['station_id']])
            
        ## start building the bike status JSON file. Will need to add more information from 2nd feed below
        ## convert Epoch time to readable for display. Keep Epoch last updated in dictionary for comparison later
        json_list_bikes_avail.append({"ID": station['station_id'],\
                "Bikes Available": station['num_bikes_available'],\
                "Updated": time.ctime(station['last_reported']),\
                "Total Activity": dict_bikes_master[station['station_id']]['total_activity'],\
                "color": dict_bikes_master[station['station_id']]['color']})
        
        

## now finish master dict with location and name info

for station in (json_station_locations['data']['stations']):  ## 4/27 - sort in chrono order for JSON file
## so that stations are ordered correctly in viz, new on bottom in file so plotted last

# for station in sorted(json_station_locations['data']['stations'], \
#                           key=lambda k: k['last_reported'], reverse = True):

    try:
        dict_bikes_master[station['station_id']]['name'] = shorten(station['name'])
        dict_bikes_master[station['station_id']]['lat'] = station['lat'] ## TODO do we need this info in DICT?
        dict_bikes_master[station['station_id']]['lon'] = station['lon']
        ## go ahead and convert to pixels while we are fetching lat and lon
        dict_bikes_master[station['station_id']]['y'] = lat_to_y(station['lat'])
        dict_bikes_master[station['station_id']]['x'] = lon_to_x(station['lon'])       
        
        dict_bikes_master[station['station_id']]['region_id'] = station['region_id']
        
        
        ## Build our two JSON files for visualization
        json_list_stations_locations.append({"name": shorten(station['name']),\
                        "Last Reported": dict_bikes_master[station['station_id']]['last_reported'],\
                        "ID": station['station_id'],\
                        "x": lon_to_x(station['lon']),\
                            "y": lat_to_y(station['lat'])})
    
    
    ## we just want to add to stations that are open, which is what our dict is. json_station_location has ALL the
    ## stations so we can use try/except to skip the ones which aren't in our master dict, ie. the closed ones
    except KeyError:
        pass


## TODO Put this in comprehension or combine with above

## Add station name to bikes available json. The feed only comes with the station ID so we must get the name
## from the station information JSON feed. The dict contains ALL info needed from both feeds so just iterate
## through it to get the names. Names are needed for display purposes.
for station in json_list_bikes_avail:
    
    station['name'] = dict_bikes_master[station['ID']]['name']
    

    ## write to our output file
json_list_stations_locations = sorted(json_list_stations_locations, \
                                      key=lambda k: k['Last Reported'], reverse = False)    
    
json_out('project_posits.json', json_list_stations_locations, 'positions')
json_out('project_data.json', json_list_bikes_avail, 'data')

## update our point of comparison time
## we'll use this later to only look for stations that have been updated afterwards
last_compared_time = json_bikes_avail['last_updated']

In [17]:
json_list_stations_locations

[{'ID': '21',
  'Last Reported': 1493293460,
  'name': 'Prudential Center - Belvedere St',
  'x': -52.4,
  'y': 42.86},
 {'ID': '6',
  'Last Reported': 1493293456,
  'name': 'Cambridge St at Joy St',
  'x': -13.65,
  'y': 13.27},
 {'ID': '20',
  'Last Reported': 1493293452,
  'name': 'Aquarium T Stop - 200',
  'x': 16.44,
  'y': 16.0},
 {'ID': '121',
  'Last Reported': 1493293450,
  'name': 'W Broadway at Dorchester St',
  'x': 29.2,
  'y': 62.82},
 {'ID': '190',
  'Last Reported': 1493293441,
  'name': 'Nashua Street at Red Auerbach',
  'x': -11.7,
  'y': 4.52},
 {'ID': '39',
  'Last Reported': 1493293440,
  'name': 'Washington St at Rutland St',
  'x': -33.42,
  'y': 57.33},
 {'ID': '184',
  'Last Reported': 1493293438,
  'name': 'Sidney Research Campus/ Erie Street',
  'x': -99.85,
  'y': 19.92},
 {'ID': '53',
  'Last Reported': 1493293435,
  'name': 'Beacon St at Massachusetts Ave',
  'x': -68.64,
  'y': 33.35},
 {'ID': '134',
  'Last Reported': 1493293431,
  'name': 'Boylston St a

## Update Loop Starts Here!

### So the only way to determine if a bike has been rented or returned is to compare current availability to 
### previous availability. We have our baseline from above. Now below we will fetch all the bike availability
### do math, and see changes if any. Then we will update the availability and repeat....

## Split below up into various cells as able

## See if we upload ALL the positions but only stations that have activity IF the colors will update...

In [28]:
### MAIN ###########
count = 0
json_list_bikes_avail_updated = []

## So that it doesn't run indefinitely, we will loop for ~ 2 minutes. Can run cell again without any variables
## being reset or run a few minutes later. It's just going to compare data to either the cell above 
## if this is first loop, or the last loop run here.
while count < 12:
  
    ## pause for the ttl time of the update feed. Don't want to run this right after the above cell has run
    ## so we pause at the beginning of the loop for the ttl time (set in constants). This value is hardcoded
    ## from the Hubway JSON feed and determined by Hubway. We could update less frequently if we are hitting
    ## server too often but this approximates 'real time' data 
    time.sleep(ttl)
    
    
    ## Get latest bike availability
    json_bikes_avail_updated = json_in(url_station_status)
    
    ## TODO - eventually put this in a while loop that times out after xx tries or seconds
    ## basically if the data is old sleep and try again. need a counter or something to avoid infinite loop
    
    ## feed is updated every 10 seconds and we are hitting every 10 seconds. In case we are a little early
    ## or feed is a little slow and the updated time hasn't changed, we will pause for 2 seconds and try again
    ## TODO - we will do this 30 times and then give up ### Try/Except
    while json_bikes_avail_updated['last_updated'] == last_compared_time:
        time.sleep(2)
        json_bikes_avail_updated = json_in(url_station_status)
    
    ### TODO - does this line below do anything???
    data2 = (json_bikes_avail_updated['data']['stations'])
    
    ##  display current feed time
    
    print('Current feed time is-', time.ctime(json_bikes_avail_updated['last_updated']),'\n\n')
    
#################
    ###json_list_bikes_avail_updated = [] 4/27

    #for station in json_bikes_avail_updated['data']['stations']:
        
    ## sort each update to display chronologically. Purely for display purposes
    ## this is the new JSON feed from above that we are going through chrono to see if there are changes
    ## sorting now will ensure that printing later is chronological
    
    ##http://stackoverflow.com/questions/72899/how-do-i-sort-a-list-of-dictionaries-by-values-of-the-dictionary-in-python
    for station in sorted(json_bikes_avail_updated['data']['stations'], \
                          key=lambda k: k['last_reported'], reverse = False):
    

        ## So we're looking for changes in the previous availability at stations. Instead of comparing
        ## current to previous at each station, first see if station has reported in since last update
        ## IF it hasn't then no need to do math and see if number of bikes have changed
        
    
        if (station['last_reported'] > last_compared_time):
            ## we're not going to add new stations that come online after our base case so we
            ## will exclude via try/except. Otherwise if a station comes online after we make our initial
            ## dict then we will be attempting to update a non-existent key.
            
            
            ## only need one statement in the 'try' block. If key error then use except
            ## to pass to the next if loop item, use 'else' statement to execute the rest of statements
            ## below.
            
            ## we need this assigment but are also using it as test to see if we need to ignore a new station
            try: 
                
                previous_bikes_avail = dict_bikes_master[station['station_id']]['num_bikes_available']
                
            ## key for current station in JSON feed doesn't exist in our dictionary. We will skip it
            except Exception as e:# KeyError:
                print(e)
#                 pass

            ## no exception so just finish up the loop
            else:
                
                
                ## now only deal with bikes that have changed in quantitiy, 
                ## previous is set from master dict, current is from new JSON feed
                if (previous_bikes_avail != station['num_bikes_available']):
                    current_bikes_avail = station['num_bikes_available']
                
                    ## Print functin here
                    
                    print_status_console(station)
                    

                    ## update master data
                    dict_bikes_master[station['station_id']].update(station)
                    set_color_status(dict_bikes_master[station['station_id']])
                    ## 4/25
                    ## absolute value as we are just interested in total activity
                    dict_bikes_master[station['station_id']]['total_activity'] +=\
                    abs(current_bikes_avail - previous_bikes_avail)
                    
                    ## Temp to test functionality. Will need to re-write so that all stations are written
                    ## and not just current update
                    json_list_bikes_avail_updated.append({"ID": station['station_id'],\
                        "Bikes Available": station['num_bikes_available'],\
                        "Updated": time.ctime(station['last_reported']),\
                        "Total Activity": dict_bikes_master[station['station_id']]['total_activity'],\
                        "color": dict_bikes_master[station['station_id']]['color']})




                    ########## 4/25 so is there a new JSON file yet?? Figure out today########

                    ### TODO - Figure out what to do with JSON file. Rewrite whole file and redraw? OR
                    ### write changes to new file and push that???
                    
                    ### maybe start with empty visualization and go from there
                    
                    ## start building the bike status JSON file. Will need to add more information from 2nd feed below
        ## convert Epoch time to readable for display. Keep Epoch last updated in dictionary for comparison later
#         
        
#         for station in json_list_bikes_avail:
    
#     station['name'] = dict_bikes_master[station['ID']]['name']
    


                    ## Send text message about low bikes
                    if station['num_bikes_available'] < alert_number:
                        send_alert(station['station_id'])
                        
                    
                    #### Added functionality here - put test for disabled bikes, etc....
                
            
## update our point of comparison time
    last_compared_time = json_bikes_avail_updated['last_updated']
    
    ### 4/27 another temporary to test functinality. will eventually pull all from dict where activity > 0
    for station in json_list_bikes_avail_updated:
    
        station['name'] = dict_bikes_master[station['ID']]['name']
    
    json_out('project_data.json', json_list_bikes_avail_updated, 'data')



    count +=1

Current feed time is- Thu Apr 27 08:21:01 2017 


Time reported - Thu Apr 27 08:20:42 2017
Beacon St at Arlington St
Change of -1 bikes
2 - currently available
3 - previously available

SMS message sent  Beacon St at Arlington St is low on bikes. 2 available.
Time reported - Thu Apr 27 08:20:45 2017
Roxbury Crossing T Stop -
Change of -1 bikes
0 - currently available
1 - previously available

SMS message sent  Roxbury Crossing T Stop - is low on bikes. 0 available.
Time reported - Thu Apr 27 08:20:48 2017
Harvard University Gund Hall at
Change of 1 bikes
6 - currently available
5 - previously available

Time reported - Thu Apr 27 08:20:51 2017
Washington St at Rutland St
Change of -1 bikes
8 - currently available
9 - previously available

Time reported - Thu Apr 27 08:20:57 2017
Central Square at Mass Ave
Change of 1 bikes
7 - currently available
6 - previously available

Time reported - Thu Apr 27 08:20:57 2017
359 Broadway - Broadway at
Change of -1 bikes
7 - currently available
8 - 

In [23]:
json_list_bikes_avail_updated

[{'Bikes Available': 3,
  'ID': '58',
  'Total Activity': 3,
  'Updated': 'Thu Apr 27 08:02:16 2017',
  'color': 'yellow'},
 {'Bikes Available': 6,
  'ID': '143',
  'Total Activity': 2,
  'Updated': 'Thu Apr 27 08:02:19 2017',
  'color': 'green'},
 {'Bikes Available': 12,
  'ID': '194',
  'Total Activity': 1,
  'Updated': 'Thu Apr 27 08:02:21 2017',
  'color': 'green'},
 {'Bikes Available': 4,
  'ID': '76',
  'Total Activity': 2,
  'Updated': 'Thu Apr 27 08:02:25 2017',
  'color': 'yellow'},
 {'Bikes Available': 7,
  'ID': '117',
  'Total Activity': 2,
  'Updated': 'Thu Apr 27 08:02:30 2017',
  'color': 'green'},
 {'Bikes Available': 7,
  'ID': '95',
  'Total Activity': 1,
  'Updated': 'Thu Apr 27 08:02:39 2017',
  'color': 'green'},
 {'Bikes Available': 10,
  'ID': '81',
  'Total Activity': 1,
  'Updated': 'Thu Apr 27 08:02:40 2017',
  'color': 'green'},
 {'Bikes Available': 11,
  'ID': '66',
  'Total Activity': 4,
  'Updated': 'Thu Apr 27 08:03:01 2017',
  'color': 'green'},
 {'Bikes

In [94]:
for station in dict_bikes_master:
    if dict_bikes_master[station]['total_activity'] > 0:
        print(dict_bikes_master[station]['total_activity'], dict_bikes_master[station]['name'])


1 Ruggles T Stop - Columbus
1 South Boston Library - 646
1 MIT Stata Center at Vassar
1 Back Bay T Stop -
1 Porter Square Station


In [ ]:
#### OLD!!! ##########


## look at test the response status for whichever lib I am using
# response = requests.get(url)

# if response.status_code == 200:

#     data = response.json()

## Basically run until program is interrupted
while True:
  
    ## pause for the ttl time of the update feed.
    time.sleep(ttl)
    
    
    json_bikes_avail_updated = json_in(url_station_status)
    
    ## TODO - eventually put this in a while loop that times out after xx tries or seconds
    ## basically if the data is old sleep and try again. need a counter or something to avoid infinite loop
    while json_bikes_avail_updated['last_updated'] == last_compared_time:
        time.sleep(2)
        json_bikes_avail_updated = json_in(url_station_status)
    
    print (type(json_bikes_avail_updated))
    #print (data)
    data2 = (json_bikes_avail_updated['data']['stations'])
#################
    json_list_bikes_avail_updated = []

    for station in json_bikes_avail_updated['data']['stations']:
        
        ## only compare stations that have been updated since last feed
        if station['last_reported'] > last_compared_time:
            ## we're not going to add new stations that come online after our base case so we
            ## will exclude via try/except.
            try:
                print(station['name'])
                print(dict_bikes_master[station['station_id']]['num_bikes_available'] - station['num_bikes_available'])
                
                
                
            except:
                pass
        
        dict_bikes_master[station['station_id']] = station ## TODO -we'll just save all the date for now, pare it down later

        if station['is_renting'] == 1 and station['station_id']!= '220': ## station is open so we can plot it and update the mster dict
            ### TODO think this is where dict is resetting. think this through....
            dict_station_locations[station['station_id']].update(station)
            if station['num_bikes_available'] < 3:
                dict_station_locations[station['station_id']]['color'] = 'red'
            elif station['num_bikes_available'] < 6:           
                dict_station_locations[station['station_id']]['color'] = 'yellow'
            else:
                dict_station_locations[station['station_id']]['color'] = 'green'


            json_list_bikes_avail.append({"ID": station['station_id'],\
                            "Bikes Available": station['num_bikes_available'],\
                            "color": dict_station_locations[station['station_id']]['color'],\
                                       "name": shorten(dict_station_locations[station['station_id']]['name'])})
        ## TODO look at reversing order dict is made so we don't have to delete items
        ## remove it from the mster dict. it was added because we had to make dict with lat/lon feed first
        ## not feed with station status
#         elif station['station_id'] != '220':
#             del dict_station_locations[station['station_id']] 

        ## write to our output file
    json_out('project_data.json', json_list_bikes_avail, 'data')
    
    
    
##################    
    ##http://stackoverflow.com/questions/72899/how-do-i-sort-a-list-of-dictionaries-by-values-of-the-dictionary-in-python
    for station in sorted(data2, key=lambda k: k['last_reported'], reverse = True):
    #for station in data2:
        if station['is_renting'] == 1 and (station['last_reported'] > last_compared_time):
            print(dict_station_locations[station['station_id']]['name'], 'Bikes available', station['num_bikes_available'], \
                  'Updated',time.ctime(station['last_reported']))
            print("Change of ",station['num_bikes_available']- \
                  dict_station_locations[station['station_id']]['num_bikes_available'],'bikes')
            
            ###TODO this doesn't appear to be working, think i"m geting cumlative frombeginning
            ## update current bikes availbe for current station
            ## master DICT is being reset somewhere. Look at Aquarium below in output
            print('debug',dict_station_locations[station['station_id']]['num_bikes_available'])
            print('debug',station['num_bikes_available'])

            dict_station_locations[station['station_id']]['num_bikes_available'] = station['num_bikes_available']
            
            print('debug',dict_station_locations[station['station_id']]['num_bikes_available'])


            
            
            if station['num_bikes_available'] < alert_number:

                ## for cleanliness in message call
                lat = dict_station_locations[station['station_id']]['lat']
                lon = dict_station_locations[station['station_id']]['lon']
                stat = dict_station_locations[station['station_id']]['name']
                send_alert()


    ## update our point of comparison time
    last_compared_time = data['last_updated']


http://stackoverflow.com/questions/474528/what-is-the-best-way-to-repeatedly-execute-a-function-every-x-seconds-in-python

### code below is side code that eventually needs to be integrated. removes the off line statins and extends out the plotting are via changes to the pixel width in the functions

In [ ]:
def long_to_x(long):
    return round((70)*(90 +(float(long)+71.05)*10000) / 180,2)

def lat_to_y(lat):
    return round((95)*(180 -(float(lat)-42.35)*10000) / 360,2)


json_list_stations_locations = []
for station in dict_station_locations:
    if dict_station_locations[station]['is_renting'] == 1:
  
        if dict_station_locations[station]['station_id'] != '220':
            json_list_stations_locations.append({"name": shorten(dict_station_locations[station]['name']),\
                            "ID": dict_station_locations[station]['station_id'],\
                            "x": long_to_x(dict_station_locations[station]['lon']),\
                                "y": lat_to_y(dict_station_locations[station]['lat'])})

    ## write to our output file
json_out('project_posits.json', json_list_stations_locations, 'positions')
    

In [ ]:
for station in dict_station_locations:
    print(dict_station_locations[station],'\n')

In [ ]:
for station in dict_station_locations:
    #print(dict_station_locations[station['station_id']])
    if dict_station_locations[station]['name'] == 'Davis Square':
        print(dict_station_locations[station]['num_bikes_available'])

#print(dict_station_locations['140'])